In [1]:
import numpy as np

In [2]:
def write_header_lines(file, layer_height, line_width, layer_count, mesh, speed=1200):
    with open(file, 'w') as f:
        header_lines = [
            f";layer_height = {layer_height}",
            f"\n;line_width = {line_width}",
            f"\n;layer_count = {layer_count}",
            f"\n;mesh = {mesh}"
        ]
        
        initialize_lines = [
            "\nG21 ;start of the code",
            "\nG1 Z15 F300",
            "\nG28 X0 Y0 ;Home",
            "\nG92 X0 Y0 ;Consider this as current",
            "\nG0 X50 Y50 F3000 ;Go-to Offset",  
            "\nG92 X0 Y0 ;Reset",
            "\n",
            "\nG0 F3600 X-5.8 Y-14.285 Z0.3",
            "\nM7",
            "\n\n"
        ]
        
        f.writelines(header_lines)
        f.writelines(initialize_lines)
        
        
def writ_finish_lines(file):
    with open(file, 'a') as f:
        finish_lines = [
            "\n\n;Finish",
            "\nM9",
            "\nG92 E1"
            "\nG1 E-1 F300"
            "\nG28 ;Home",
        ]
        f.writelines(finish_lines)

In [21]:
def rotate_45_degree(xy):
    x, y = xy
    x_new = x * np.cos(np.pi/4) - y * np.sin(np.pi/4)
    y_new = x * np.sin(np.pi/4) + y * np.cos(np.pi/4)
    return np.array([x_new, y_new])

def rotate_xy(xy, times):
    for _ in range(times):
        xy = rotate_45_degree(xy)
    return xy

def write_one_line_with_rotate(lines, xy, rotate_times):
    xy = rotate_xy(xy, rotate_times)
    x, y = xy
    lines.append(f"\nG1 X{x} Y{y}")
    # return lines

In [42]:
def draw_layer(file, x_barrier):
    lines = []
    
    for i in range (8):
        xy = np.array([-5.923, -14.301])
        if (x_barrier < -5.5) or (x_barrier > 5.5):
            xy += [10.546,0]
            write_one_line_with_rotate(lines, xy, i)
            xy += [0, 0.4]
            write_one_line_with_rotate(lines, xy, i)
            xy += [-9.646,0]
            write_one_line_with_rotate(lines, xy, i)
            xy += [0, 0.4]
            write_one_line_with_rotate(lines, xy, i)
            xy += [10.046,0]
            write_one_line_with_rotate(lines, xy, i)
            xy += [0, -0.8]
            write_one_line_with_rotate(lines, xy, i)
            xy += [0.9,0]
            write_one_line_with_rotate(lines, xy, i)
            
        else:
            xy += [x_barrier-0.5-0.2+5.923, 0]
            write_one_line_with_rotate(lines, xy, i)
            xy += [0, 0.4]
            write_one_line_with_rotate(lines, xy, i)
            xy += [-(x_barrier-0.5-0.2+5.923-0.9),0]
            write_one_line_with_rotate(lines, xy, i)
            xy += [0, 0.4]
            write_one_line_with_rotate(lines, xy, i)
            xy += [10.046,0]
            write_one_line_with_rotate(lines, xy, i)
            xy += [0, -0.4]
            write_one_line_with_rotate(lines, xy, i)
            xy += [-(-x_barrier-0.5-0.2+5.923-0.9),0]
            write_one_line_with_rotate(lines, xy, i)
            xy += [0, -0.4]
            write_one_line_with_rotate(lines, xy, i)
            xy += [-x_barrier-0.5-0.2+5.923,0]
            write_one_line_with_rotate(lines, xy, i)
            
    with open(file, 'a') as f:
        f.writelines(lines)

In [45]:
layer_height = 0.4
line_width = 0.5
layer_count = 60
mesh = 'only wall'
side_length = 12
wall_layer_count = 0
file = '7.17/string_'+str(layer_count)+'.gcode'
write_header_lines(file, layer_height, 0.4, layer_count, mesh)

z = 0.3
for layer in range (layer_count):
    barrier_x = -5.8 + 11.6*layer/(layer_count-1)
    # print(barrier_x)
    draw_layer(file, barrier_x)
    z += 0.4
    line = f"\nG1 Z{z}"
    with open(file, 'a') as f:
        f.write(line)

: 